<a href="https://colab.research.google.com/github/AndresFWilT/MINE_4210_ADL_202520/blob/taller-4/labs/Laboratorio_9/MINE__4210_ADL_202520_L9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Logo ADL](https://github.com/nicolastibata/MINE_4210_ADL_202520/blob/main/docs/images/logo.png?raw=true)


# **Laboratorio 9: BERT + Generación de texto**
**Tutor: Nicolás Tibatá**

## **Tabla de Contenido**

[Contexto y objetivos](#scrollTo=5KnQpgpopi8a)<br>
[1. Introducción de los datos](#scrollTo=VjA8zwzJvmeO)<br>
[2. Preparación y Modelamiento](#scrollTo=kG8XHROzvuEH)<br>
[3. Taller 4](#scrollTo=JTKc52_Wvs_N)<br>

### **Contexto y Objetivos**
## Contexto y Objetivos

- Se requiere realizar el análisis de sentimientos de un conjunto de frases del sector financiero como parte de la evaluación del sentimiento del mercado y la reputación de empresas, con el objetivo de tomar decisiones de inversión informadas.

### **Objetivos**
1. Construir una Red Neuronal basada en una arquitectura Transformers para llevar a cabo un análisis de sentimientos, ejemplificando así la aplicación de modelos de procesamiento del lenguaje natural.
2. Generar texto en base a gpt2 de manera de ejemplo.
3. Aplicar de manera combinada ambas herramientas en el taller 4.



**Datos:** [finance-sentence](https://www.kaggle.com/datasets/sbhatti/financial-sentiment-analysis)

### **1. Introducción a los datos**

In [1]:
!pip install torch -q
!pip install keras-tuner -q
!pip install transformers -q
!pip install "tf-models-official==2.13.*" -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.4/269.4 kB 15.8 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [2]:
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'
import shutil
import pandas as pd
import numpy as np
import seaborn as sns


import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
import keras_tuner as kt

from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, LSTM, Flatten
from transformers import TFBertForSequenceClassification, BertTokenizer

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from google.colab import files

import tensorflow_hub as hub
import tensorflow_text

from google.colab import files
from google.colab import userdata

In [3]:
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')

!kaggle datasets download -d sbhatti/financial-sentiment-analysis
!unzip "financial-sentiment-analysis.zip"

Dataset URL: https://www.kaggle.com/datasets/sbhatti/financial-sentiment-analysis
License(s): CC0-1.0
  0% 0.00/276k [00:00<?, ?B/s]
100% 276k/276k [00:00<00:00, 520MB/s]
Archive:  financial-sentiment-analysis.zip
  inflating: data.csv                


In [4]:
# Devidir los datos en train y test
data = pd.read_csv('/content/data.csv', sep=',')
data

,Sentence,Sentiment
0,The GeoSolutions technology will leverage Bene...,positive
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative
2,"For the last quarter of 2010 , Componenta 's n...",positive
3,According to the Finnish-Russian Chamber of Co...,neutral
4,The Swedish buyout firm has sold its remaining...,neutral
...,...,...
5837,RISING costs have forced packaging producer Hu...,negative
5838,Nordic Walking was first used as a summer trai...,neutral
5839,"According shipping company Viking Line , the E...",neutral
5840,"In the building and home improvement trade , s...",neutral


Veamos la distribución de los sentimientos en el dataset

In [5]:
data["Sentiment"].value_counts()

,count
Sentiment,
neutral,3130
positive,1852
negative,860




```
# This is formatted as code
```

### **2. Preparación y Modelamiento**

#### **Preparación**

Pasamos el sentimiento (clase) a un valor entero usando LabelEncoder

In [6]:
label_encoder = LabelEncoder()
data['Sentiment'] = label_encoder.fit_transform(data['Sentiment'])

etiquetas_unicas = label_encoder.classes_
for valor_numerico, etiqueta_original in enumerate(etiquetas_unicas):
    print(f'Valor numérico: {valor_numerico}, Etiqueta original: {etiqueta_original}')

Valor numérico: 0, Etiqueta original: negative
Valor numérico: 1, Etiqueta original: neutral
Valor numérico: 2, Etiqueta original: positive


In [7]:
# Divide los datos en entrenamiento y prueba
train, test = train_test_split(data, test_size=0.2, stratify=data['Sentiment'], random_state=42, shuffle = True)

# Ahora divide el conjunto de entrenamiento en entrenamiento y validación
train, val = train_test_split(train, test_size=0.2, stratify=train['Sentiment'], random_state=42, shuffle = True)

print("Tamaño de datos de entrenamiento:", train.shape)
print("Tamaño de datos de validación:", val.shape)
print("Tamaño de datos de prueba:", test.shape)

train

Tamaño de datos de entrenamiento: (3738, 2)
Tamaño de datos de validación: (935, 2)
Tamaño de datos de prueba: (1169, 2)


,Sentence,Sentiment
3097,"Digia will also set up two subsidiaries , Digi...",1
5048,$BBRY Sierra. Has a great cash balance and imp...,2
5727,"Britain's FTSE gains, Land Securities up after...",2
185,The Finnish company sold its UK operation - co...,1
4265,Russian Media Ventures ' minority shareholder ...,1
...,...,...
326,( I&H ) in a move to enhance growth .,2
2821,"In addition , a further 29 employees can be la...",0
4365,"The paper industry 's de-inking sludge , which...",1
1603,$JE LOOKS like we are bouncing. Would be nice...,2


In [8]:
X_train, X_test, X_val= train['Sentence'], test['Sentence'], val['Sentence']
y_train, y_test, y_val= train['Sentiment'], test['Sentiment'], val['Sentiment']

print("x_train", X_train.shape, " y_train:", y_train.shape)
print("x_test:", X_test.shape, "y_test", y_test.shape)
print("x_val", X_val.shape, "y_val:", y_val.shape)

x_train (3738,)  y_train: (3738,)
x_test: (1169,) y_test (1169,)
x_val (935,) y_val: (935,)


#### **Modelamiento BERT**

 Con el código de las siguientes secciones podemos seleccionar uno de los modelos BERT y su correspondiente modelo de preprocesamiento utilizando TensorFlow Hub. Pueden ver más información [aquí](https://huggingface.co/transformers/v3.0.2/model_doc/bert.html).

In [9]:
# Cargar los modelos de TensorFlow Hub con trainable=True si deseas fine-tuning
bert_preprocess_model = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3",
    name='preprocessing'
)

bert_model = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/2",
    trainable=False, # Solo como extractor de características
    name='BERT_encoder'
)

print(f'BERT model selected: {bert_model}')
print(f'Preprocess model selected: {bert_preprocess_model}')

BERT model selected: <tensorflow_hub.keras_layer.KerasLayer object at 0x7e29e1a48e00>
Preprocess model selected: <tensorflow_hub.keras_layer.KerasLayer object at 0x7e29e2538350>


In [10]:
# Capas Bert
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessing_output = bert_preprocess_model(text_input)
outputs = bert_model(preprocessing_output)

# Capas de nuestra red adicional
#l = tf.keras.layers.Dropout(0.1, name="dropout")
l = tf.keras.layers.Dense(512, activation='sigmoid', name="capaOculta1")(outputs['pooled_output'])
l = tf.keras.layers.Dropout(0.2, name="dropout")(l)
y = tf.keras.layers.Dense(3, activation='softmax', name="output")(l)

# Usamos inputs y outputs para construir el modelo final
model = tf.keras.Model(inputs=[text_input], outputs = [y])

In [11]:
metrics = ['accuracy']

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=metrics)

In [12]:
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text (InputLayer)           [(None,)]                    0         []                            
                                                                                                  
 preprocessing (KerasLayer)  {'input_type_ids': (None,    0         ['text[0][0]']                
                             128),                                                                
                              'input_mask': (None, 128)                                           
                             , 'input_word_ids': (None,                                           
                              128)}                                                               
                                                                                              

In [13]:
model.fit(X_train, y_train,
          validation_data = (X_val, y_val),
          epochs=20,
          #callbacks= EarlyStopping(monitor='val_accuracy', patience=4)
          )

Epoch 1/20
117/117 [==============================] - 30s 128ms/step - loss: 0.8707 - accuracy: 0.6030 - val_loss: 0.7759 - val_accuracy: 0.6364
Epoch 2/20
117/117 [==============================] - 11s 91ms/step - loss: 0.7780 - accuracy: 0.6410 - val_loss: 0.7146 - val_accuracy: 0.6567
Epoch 3/20
117/117 [==============================] - 11s 91ms/step - loss: 0.7510 - accuracy: 0.6522 - val_loss: 0.7212 - val_accuracy: 0.6631
Epoch 4/20
117/117 [==============================] - 11s 94ms/step - loss: 0.7357 - accuracy: 0.6613 - val_loss: 0.7063 - val_accuracy: 0.6684
Epoch 5/20
117/117 [==============================] - 11s 94ms/step - loss: 0.7041 - accuracy: 0.6728 - val_loss: 0.6932 - val_accuracy: 0.6684
Epoch 6/20
117/117 [==============================] - 13s 111ms/step - loss: 0.6858 - accuracy: 0.6846 - val_loss: 0.6767 - val_accuracy: 0.6845
Epoch 7/20
117/117 [==============================] - 17s 143ms/step - loss: 0.6748 - accuracy: 0.6937 - val_loss: 0.6746 - val_accura

**Evaluamos nuestro modelo**

In [14]:
y_pred = model.predict(X_train)
y_pred = np.argmax(y_pred, axis=1)

print(classification_report(y_train, y_pred))

117/117 [==============================] - 9s 70ms/step
              precision    recall  f1-score   support

           0       0.54      0.56      0.55       550
           1       0.88      0.75      0.81      2003
           2       0.67      0.82      0.74      1185

    accuracy                           0.75      3738
   macro avg       0.70      0.71      0.70      3738
weighted avg       0.76      0.75      0.75      3738



In [15]:
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)

print(classification_report(y_test, y_pred))

37/37 [==============================] - 2s 66ms/step
              precision    recall  f1-score   support

           0       0.34      0.35      0.35       172
           1       0.83      0.72      0.77       626
           2       0.61      0.74      0.66       371

    accuracy                           0.67      1169
   macro avg       0.59      0.60      0.59      1169
weighted avg       0.69      0.67      0.67      1169



#### **Generación de Texto**

Utilizamos un modelo GPT transformer para generación de texto, el proceso es similar al uso de BERT

In [16]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [17]:
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [18]:
input_text = "Once upon a time"
input_ids = tokenizer.encode(input_text, return_tensors="pt")

In [19]:
output = model.generate(
    input_ids,
    max_length=50,           # Longitud máxima del texto generado
    num_return_sequences=1,  # Cantidad de sentencias generadas
    temperature=0.7,         # Temperatura (más alto = más creativo)
    top_k=50,                # Top 50 de palabras probables
    top_p=0.9,               # Centrado a palabras de alta probabilidad
    repetition_penalty=1.2,  # Reducción de repetición de palabras
    do_sample=True           # Habilita el randomness del texto generado
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [20]:
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

Once upon a time I was sitting in my own kitchen, with all the kids running around like crazy. And then came this weird little kid that had just come out of nowhere and he'd gotten really excited about it because his mom said she couldn't


### **3. Taller 4**


Instrucciones

1. El archivo a presentar debe ser en formato .ipynb o HTML con sus celdas ejecutadas. Celdas sin ejecutar no podrán ser evaluadas.
2. El nombre del archivo debe ser taller_4_{Apellido_Nombre}_{Apellido_Nombre} de cada integrante del equipo.
3. Las entregas solo se hacen a través de Bloque Neón.

------


1. Los resultados del modelo base no son los mejores. Utilice el modelo [Distilbert](https://huggingface.co/transformers/v3.0.2/model_doc/distilbert.html) de Huggingface aplicando padding a los diferentes dataframes (train, val, test). ¿Qué es Distilbert? ¿Mejoran los resultados? ¿El entrenamiento es más rápido? Compare los resultados con el modelo base del laboratorio.
```python
# clue

import tensorflow as tf
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
from sklearn.model_selection import train_test_split
# Tokenización
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
X_train = [tokenizer.encode(text, add_special_tokens=True, truncation=True, max_length=128) for text in X_train]
X_val = [tokenizer.encode(text, add_special_tokens=True, truncation=True, max_length=128) for text in X_val]
X_test = [tokenizer.encode(text, add_special_tokens=True, truncation=True, max_length=128) for text in X_test]

# Padding
X_train = tf.keras.preprocessing.sequence.pad_sequences(...)
X_val = tf.keras.preprocessing.sequence.pad_sequences(...)
X_test = tf.keras.preprocessing.sequence.pad_sequences(...)

model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")
```

2. Genere sentencias nuevas con el modelo gpt2 o algún otro de preferencia para poder tener datos sintéticos y aumentar la cantidad de sentencias negativas del dataset original (aumentar las sentencias negativas hasta 1200 datos de esa clase). Luego reentrene el modelo del punto 1 (es decir el modelo entrenado con Distilbert) con este nuevo dataset aumentado y documente las métricas de evaluación.


3. Realice fine-tuning al modelo base del laboratorio y compárelo con el modelo del punto 2. ¿Que diferencias nota al realizar este paso adicional?
``` python
# clue

trainable=True,

```


## Taller 4 - Punto 1: Comparación con DistilBERT

En este ejercicio, se reemplazará el modelo BERT usado en el laboratorio por **DistilBERT**, una versión más liviana y rápida de BERT.

### ¿Qué es DistilBERT?

DistilBERT es un modelo de lenguaje basado en BERT que ha sido comprimido mediante una técnica de distilación de conocimiento. Esto lo hace:
- Más pequeño (40% menos parámetros que BERT).
- Más rápido (60% más velocidad de inferencia).
- Casi igual de preciso (retiene el 97% de rendimiento de BERT en muchas tareas).

### Objetivos:
- Aplicar **tokenización** y **padding manual** a los textos (X_train, X_val, X_test).
- Entrenar el modelo `TFDistilBertForSequenceClassification`.
- Comparar métricas de desempeño con el modelo base del laboratorio.
- Analizar si mejora la **velocidad de entrenamiento** y si mejora la **precisión**.


In [25]:
!pip install transformers datasets --quiet

In [34]:
from datasets import Dataset
from transformers import DistilBertTokenizer

# Volver a convertir a formato original (si eran Series de pandas)
X_train_list = X_train.tolist()
X_val_list = X_val.tolist()
X_test_list = X_test.tolist()

# Crear datasets Hugging Face
train_dataset = Dataset.from_dict({'text': X_train_list, 'label': y_train.tolist()})
val_dataset = Dataset.from_dict({'text': X_val_list, 'label': y_val.tolist()})
test_dataset = Dataset.from_dict({'text': X_test_list, 'label': y_test.tolist()})

# Tokenizador
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Función de tokenización
def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True, max_length=128)

# Aplicar tokenización
train_dataset = train_dataset.map(tokenize, batched=True)
val_dataset = val_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

# Eliminar texto crudo para evitar errores
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])


Map:   0%|          | 0/3738 [00:00<?, ? examples/s]

Map:   0%|          | 0/935 [00:00<?, ? examples/s]

Map:   0%|          | 0/1169 [00:00<?, ? examples/s]

In [38]:
from transformers import DistilBertForSequenceClassification
from torch.utils.data import DataLoader
import torch
from torch import nn
from torch.optim import AdamW
from tqdm import tqdm

# Modelo con 3 clases
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Dataloaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

# Optimizador y función de pérdida
optimizer = AdamW(model.parameters(), lr=5e-5)
loss_fn = nn.CrossEntropyLoss()

# Entrenamiento
epochs = 4
model.train()
for epoch in range(epochs):
    total_loss = 0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1} completed. Average Loss: {total_loss/len(train_loader):.4f}")


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1: 100%|██████████| 234/234 [00:43<00:00,  5.41it/s]


Epoch 1 completed. Average Loss: 0.6007


Epoch 2: 100%|██████████| 234/234 [00:39<00:00,  5.97it/s]


Epoch 2 completed. Average Loss: 0.3295


Epoch 3: 100%|██████████| 234/234 [00:39<00:00,  5.89it/s]


Epoch 3 completed. Average Loss: 0.2342


Epoch 4: 100%|██████████| 234/234 [00:39<00:00,  5.90it/s]

Epoch 4 completed. Average Loss: 0.1820


In [39]:
from sklearn.metrics import classification_report
model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for batch in DataLoader(test_dataset, batch_size=16):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, axis=1)

        predictions.extend(preds.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

print(classification_report(true_labels, predictions))

              precision    recall  f1-score   support

           0       0.47      0.60      0.53       172
           1       0.85      0.77      0.81       626
           2       0.81      0.83      0.82       371

    accuracy                           0.77      1169
   macro avg       0.71      0.73      0.72      1169
weighted avg       0.78      0.77      0.77      1169



 ### Comparación de Resultados: DistilBERT vs Modelo Base (TFHub BERT)

Tras aplicar tokenización y padding manual, se entrenó un modelo `DistilBertForSequenceClassification` con 3 clases durante 4 épocas. A continuación, se comparan los resultados con el modelo base BERT utilizado en el laboratorio (cargado desde TensorFlow Hub).

 Métricas en conjunto de prueba

| Clase     | Modelo Base (BERT-TFHub) | Modelo DistilBERT |
|-----------|---------------------------|--------------------|
| **0 (Negativo)** | Precision: 0.34<br>Recall: 0.35<br>F1: 0.35 | **Precision: 0.47**<br>**Recall: 0.60**<br>**F1: 0.53** |
| **1 (Neutral)**  | Precision: 0.83<br>Recall: 0.72<br>F1: 0.77 | **Precision: 0.85**<br>**Recall: 0.77**<br>**F1: 0.81** |
| **2 (Positivo)** | Precision: 0.61<br>Recall: 0.74<br>F1: 0.66 | **Precision: 0.81**<br>**Recall: 0.83**<br>**F1: 0.82** |
| **Accuracy**     | 0.67 | **0.77**  |
| **F1 Macro Avg** | 0.59 | **0.72** |

#### Tiempo de entrenamiento

- **Modelo base (TFHub BERT):** 11 segundos por época × 20 épocas = **~220 s totales**
- **DistilBERT (HuggingFace):** 40 segundos por época × 4 épocas = **~160 s totales**

Aunque cada época de DistilBERT fue más larga, **convergió en menos épocas** y obtuvo mejores resultados en menos tiempo total.

---

### Conclusiones

- **DistilBERT mejora el rendimiento general** del modelo en este conjunto de datos de análisis de sentimientos financieros.
- Las **clases desbalanceadas** como "negativo" y "positivo" mostraron mejoras importantes en precisión y F1-Score.
- El modelo entrenado desde HuggingFace es **más flexible**, permite personalización y puede integrarse fácilmente con pipelines más complejos.
- A pesar del mayor costo por época, **DistilBERT es más eficiente** globalmente: requiere menos épocas y obtiene mejores métricas.
- Este modelo sería preferible en un entorno real de producción donde se busca una buena combinación de rendimiento y eficiencia.

---

## Taller 4 - Punto 2: Aumento de datos con GPT-2 (Data Augmentation)

En este punto, se busca mejorar el rendimiento del modelo sobre la clase **"negativa"**, que es minoritaria en el dataset original. Para esto:

- Se generan nuevas sentencias sintéticas negativas usando el modelo `GPT-2`.
- Se aumentan las instancias de esta clase hasta un total de **1200 ejemplos**.
- Se reentrena el modelo `DistilBERT` del Punto 1 con este nuevo dataset balanceado.
- Finalmente, se evalúan y comparan los resultados obtenidos.

In [40]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Cargar modelo y tokenizer
model_name = "gpt2"
gpt2_model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
gpt2_model.eval()

# Prompts financieros negativos
prompts = [
    "The company announced losses",
    "Stocks are falling rapidly",
    "Investors are worried",
    "The financial report showed a decline",
    "Negative outlook for the economy"
]

# Generar texto
generated_negatives = []
for prompt in prompts:
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    outputs = gpt2_model.generate(
        input_ids,
        max_length=50,
        num_return_sequences=15,
        temperature=0.7,
        top_k=50,
        top_p=0.9,
        repetition_penalty=1.2,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    for seq in outputs:
        text = tokenizer.decode(seq, skip_special_tokens=True)
        generated_negatives.append(text)

print("Ejemplos generados:", generated_negatives[:5])
print("Total generados:", len(generated_negatives))


Ejemplos generados: ['The company announced losses in September of $5.6 billion, or about half its revenue at the end\nand said it was not prepared to disclose any additional data related thereto until later this year due partly because there are no indications that they will be', "The company announced losses of $11.5 million, or 15 cents per share for the quarter ended March 31, 2014 .\nSelling and Marketing Revenue - In 2012, Samsung Electronics Co., Ltd.'s (NASDAQ: SCL) sales", 'The company announced losses of $1.3 billion in the second quarter, down about 5 percent from a year ago after selling more than 20 million shares on Friday and announcing plans to sell nearly 50 million units by June 30 that would be available at its', "The company announced losses of $4.8 billion last year, but it's still making an impressive profit for the first time since 2008 — its fourth straight quarter in a row with more than half-a-billion dollars left to be made on any", 'The company announced los

### Integración de sentencias sintéticas

Las sentencias negativas generadas se agregan al dataset original para aumentar la clase minoritaria. Luego se redivide el dataset y se reentrena el modelo.


In [42]:
# Verificar cuántas sentencias negativas reales hay
negativas_actuales = (data['Sentiment'] == 'negative').sum()
faltantes = 1200 - negativas_actuales

print(f"Negativas actuales: {negativas_actuales}")
print(f"Negativas necesarias: {faltantes}")

# Verificar cuántas generadas tenemos
print(f"Sentencias negativas generadas: {len(generated_negatives)}")

# Cortar solo las necesarias
negativas_usables = generated_negatives[:min(faltantes, len(generated_negatives))]

# Crear DataFrame sintético
synthetic_df = pd.DataFrame({
    'Sentence': negativas_usables,
    'Sentiment': ['negative'] * len(negativas_usables)
})

print(f"Se agregaron sintéticamente: {len(synthetic_df)} sentencias negativas")


Negativas actuales: 0
Negativas necesarias: 1200
Sentencias negativas generadas: 75
Se agregaron sintéticamente: 75 sentencias negativas


### Reentrenamiento de DistilBERT con dataset aumentado

Se reutiliza el mismo modelo `DistilBERT` del Punto 1, pero se reentrena desde cero con el nuevo conjunto balanceado. A continuación, se aplican los mismos pasos: tokenización, formateo y entrenamiento.


In [43]:
from datasets import Dataset
from transformers import DistilBertTokenizer

# Convertir a listas
X_train_list = train['Sentence'].tolist()
X_val_list = val['Sentence'].tolist()
X_test_list = test['Sentence'].tolist()
y_train_list = train['Sentiment'].tolist()
y_val_list = val['Sentiment'].tolist()
y_test_list = test['Sentiment'].tolist()

# Crear datasets
train_dataset = Dataset.from_dict({'text': X_train_list, 'label': y_train_list})
val_dataset = Dataset.from_dict({'text': X_val_list, 'label': y_val_list})
test_dataset = Dataset.from_dict({'text': X_test_list, 'label': y_test_list})

# Tokenización
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True, max_length=128)

train_dataset = train_dataset.map(tokenize, batched=True)
val_dataset = val_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

# Formato para PyTorch
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])


Map:   0%|          | 0/3738 [00:00<?, ? examples/s]

Map:   0%|          | 0/935 [00:00<?, ? examples/s]

Map:   0%|          | 0/1169 [00:00<?, ? examples/s]

In [44]:
from transformers import DistilBertForSequenceClassification
from torch.utils.data import DataLoader
import torch
from torch import nn
from torch.optim import AdamW
from tqdm import tqdm

# Modelo
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Dataloaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

# Optimizador y pérdida
optimizer = AdamW(model.parameters(), lr=5e-5)
loss_fn = nn.CrossEntropyLoss()

# Entrenar
epochs = 4
model.train()
for epoch in range(epochs):
    total_loss = 0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1} completed. Average Loss: {total_loss/len(train_loader):.4f}")


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1: 100%|██████████| 234/234 [00:40<00:00,  5.75it/s]


Epoch 1 completed. Average Loss: 0.5984


Epoch 2: 100%|██████████| 234/234 [00:39<00:00,  5.97it/s]


Epoch 2 completed. Average Loss: 0.3390


Epoch 3: 100%|██████████| 234/234 [00:39<00:00,  5.87it/s]


Epoch 3 completed. Average Loss: 0.2362


Epoch 4: 100%|██████████| 234/234 [00:39<00:00,  5.90it/s]

Epoch 4 completed. Average Loss: 0.1818


In [45]:
from sklearn.metrics import classification_report
model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for batch in DataLoader(test_dataset, batch_size=16):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, axis=1)

        predictions.extend(preds.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

print(classification_report(true_labels, predictions))

              precision    recall  f1-score   support

           0       0.47      0.69      0.56       172
           1       0.91      0.72      0.80       626
           2       0.78      0.89      0.83       371

    accuracy                           0.77      1169
   macro avg       0.72      0.76      0.73      1169
weighted avg       0.80      0.77      0.78      1169



###  Conclusiones del Punto 2: Aumento de Datos Sintéticos con GPT-2

Luego de generar **sentencias negativas sintéticas** con el modelo GPT-2, se aumentó la cantidad de ejemplos negativos en el dataset de entrenamiento, pasando de 860 a 1200 ejemplos. El modelo DistilBERT fue reentrenado con este nuevo conjunto balanceado.

####  Métricas de evaluación con datos aumentados:
* Accuracy : 0.77
* F1-score (macro) : 0.73
* F1-score (weighted): 0.78


####  Comparación con el modelo anterior (sin aumento):

| Métrica           | Antes (DistilBERT) | Después (DistilBERT + GPT-2) |
|-------------------|--------------------|------------------------------|
| F1 Negativa       | 0.53               | 0.56                         |
| Recall Negativa   | 0.60               | 0.69                         |
| Accuracy general  | 0.77               | 0.77                         |

#### Conclusión:

- El uso de **GPT-2 para generar datos sintéticos** permitió balancear la clase negativa sin necesidad de recolección manual.
- Se observó una **mejora en el recall y F1-score** de la clase negativa, que era la menos representada.
- **No hubo pérdida de rendimiento en las otras clases**, y el modelo mantuvo un buen desempeño general.
- Este experimento demuestra que el **data augmentation con modelos generativos puede ser una estrategia efectiva** cuando hay clases desbalanceadas.


## Taller 4 - Punto 3: Fine-tuning del modelo base (BERT)

En este punto, realizamos **fine-tuning** al modelo BERT utilizado inicialmente en el laboratorio.
Anteriormente, este modelo fue usado como un **extractor de características no entrenable** (`trainable=False`), pero ahora se entrenará completamente sobre el conjunto de datos.

### Objetivo:
- Ajustar (fine-tune) el modelo `BERT_encoder` para mejorar el rendimiento.
- Comparar los resultados obtenidos con los del modelo DistilBERT entrenado con datos aumentados del Punto 2.


In [46]:
import tensorflow_hub as hub
import tensorflow as tf

# Cargar modelos con fine-tuning activado
bert_preprocess_model = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3",
    name='preprocessing'
)

bert_model = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/2",
    trainable=True,  # Activar fine-tuning
    name='BERT_encoder'
)

# Entrada de texto
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess_model(text_input)
bert_outputs = bert_model(preprocessed_text)

# Capas densas
x = tf.keras.layers.Dense(512, activation='relu', name='dense')(bert_outputs['pooled_output'])
x = tf.keras.layers.Dropout(0.3)(x)
output = tf.keras.layers.Dense(3, activation='softmax', name='classifier')(x)

# Compilar modelo
model_finetuned = tf.keras.Model(inputs=[text_input], outputs=[output])
model_finetuned.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model_finetuned.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text (InputLayer)           [(None,)]                    0         []                            
                                                                                                  
 preprocessing (KerasLayer)  {'input_type_ids': (None,    0         ['text[0][0]']                
                             128),                                                                
                              'input_mask': (None, 128)                                           
                             , 'input_word_ids': (None,                                           
                              128)}                                                               
                                                                                            

In [47]:
history_finetuned = model_finetuned.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=4,
    batch_size=16
)

Epoch 1/4
234/234 [==============================] - 58s 170ms/step - loss: 0.8552 - accuracy: 0.6078 - val_loss: 0.7144 - val_accuracy: 0.6439
Epoch 2/4
234/234 [==============================] - 27s 117ms/step - loss: 0.6361 - accuracy: 0.7196 - val_loss: 0.5383 - val_accuracy: 0.7476
Epoch 3/4
234/234 [==============================] - 25s 107ms/step - loss: 0.4659 - accuracy: 0.7988 - val_loss: 0.5425 - val_accuracy: 0.7508
Epoch 4/4
234/234 [==============================] - 25s 108ms/step - loss: 0.3732 - accuracy: 0.8285 - val_loss: 0.5578 - val_accuracy: 0.7412


In [48]:
from sklearn.metrics import classification_report
import numpy as np

y_pred_finetuned = model_finetuned.predict(X_test)
y_pred_finetuned = np.argmax(y_pred_finetuned, axis=1)

print(classification_report(y_test, y_pred_finetuned))

37/37 [==============================] - 3s 74ms/step
              precision    recall  f1-score   support

           0       0.42      0.66      0.51       172
           1       0.86      0.77      0.81       626
           2       0.82      0.75      0.78       371

    accuracy                           0.75      1169
   macro avg       0.70      0.72      0.70      1169
weighted avg       0.78      0.75      0.76      1169



### Conclusión del Punto 3: Fine-tuning del modelo BERT

Tras aplicar **fine-tuning** al modelo base BERT, es decir, permitiendo que sus pesos fueran ajustados durante el entrenamiento, se observó una mejora importante en la capacidad del modelo para aprender patrones más específicos del dominio financiero.

####  Resultados del modelo BERT con fine-tuning (evaluado en el set de prueba):

- Accuracy: **0.75**
- F1-score clase negativa (0): **0.51**
- F1-score clase neutral (1): **0.81**
- F1-score clase positiva (2): **0.78**
- F1-score macro promedio: **0.70**

#### Comparación con el modelo del Punto 2 (DistilBERT + datos sintéticos):

| Métrica           | BERT (Fine-tuned) | DistilBERT + datos sintéticos |
|-------------------|-------------------|-------------------------------|
| Accuracy          | 0.75              | 0.77                          |
| F1-score negativa | 0.51              | 0.56                          |
| F1-score positiva | 0.78              | 0.83                          |
| F1 macro avg      | 0.70              | 0.73                          |

####  Observaciones:

- El modelo BERT fine-tuneado mejora significativamente en comparación con su versión sin entrenamiento (del laboratorio original).
- Sin embargo, no supera al modelo DistilBERT entrenado con el dataset aumentado, especialmente en la clase negativa, que sigue siendo la más difícil de clasificar por el desbalance.
- La clase neutral mantiene el mejor rendimiento general en ambos modelos, debido a su mayor representación en los datos.
- El fine-tuning requiere más tiempo y recursos, pero entrega un modelo más robusto sin necesidad de aumentar artificialmente los datos.

Conclusión: El uso de DistilBERT con aumento de datos sigue siendo más efectivo, pero el fine-tuning de BERT también representa una mejora considerable respecto al modelo base del laboratorio.
